In [116]:
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
# Import Austin crime data and moon files
austin_data_file = 'AustinCrime.csv'
moon_data_file = 'MoonPhase.csv'

# Read the data into a pandas dataframe
austin_df = pd.read_csv(austin_data_file)
moon_df = pd.read_csv(moon_data_file)

In [118]:
# Drop Time (Universal Time) column from moon data
# moon_df = moon_df.drop(columns=["Time (Universal Time)"])

# Change Date column to datetime format
moon_df['Date'] = pd.to_datetime(moon_df['Date'])

# Display the first 5 rows of the moon data
moon_df

,Moon Phase,Date,Time (Universal Time)
0,Last Quarter,2023-01-15,02:10
1,New Moon,2023-01-21,20:53
2,First Quarter,2023-01-28,15:19
3,Full Moon,2023-02-05,18:28
4,Last Quarter,2023-02-13,16:01
...,...,...,...
495,Last Quarter,2014-12-14,12:51
496,New Moon,2014-12-22,01:36
497,First Quarter,2014-12-28,18:31
498,Full Moon,2015-01-05,04:53


In [119]:
# Check the first 5 rows of the Austin data
austin_df

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Council District,APD Sector,APD District,Clearance Status,Clearance Date,UCR Category,Category Description,Census Block Group
0,2002923330284,FAMILY DISTURBANCE,3400,N,01/29/2003 05:30,01/29/2003,530.0,11/29/2002 05:30,11/29/2002,530.0,RESIDENCE / HOME,6.0,AD,3,N,NaN,NaN,NaN,4.530341e+09
1,2003920010029,DEADLY CONDUCT,408,N,01/01/2003 00:01,01/01/2003,1.0,01/01/2003 00:01,01/01/2003,1.0,RESIDENCE / HOME,4.0,ID,4,N,NaN,13A,Aggravated Assault,4.530021e+09
2,2003920010046,BURGLARY NON RESIDENCE,502,N,01/01/2003 00:02,01/01/2003,2.0,01/01/2003 00:02,01/01/2003,2.0,COMMERCIAL / OFFICE BUILDING,2.0,DA,3,NaN,10/18/2003,220,Burglary,4.530024e+09
3,2003920010048,DEADLY CONDUCT,408,N,01/01/2003 00:03,01/01/2003,3.0,01/01/2003 00:03,01/01/2003,3.0,RESIDENCE / HOME,4.0,ID,2,C,01/02/2003,13A,Aggravated Assault,4.530402e+09
4,2003920010079,RESISTING ARREST OR SEARCH,905,N,01/01/2003 00:06,01/01/2003,6.0,01/01/2003 00:06,01/01/2003,6.0,RESIDENCE / HOME,3.0,CH,2,C,01/01/2003,NaN,NaN,4.530009e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522580,20242241547,POSSESSION OF MARIJUANA,1803,N,08/11/2024 23:12,08/11/2024,2312.0,08/11/2024 23:12,08/11/2024,2312.0,GAMBLING FACILITY / CASINO / RACE TRACK,NaN,HE,NaN,NaN,NaN,NaN,NaN,NaN
2522581,20232120660,AUTO THEFT,700,N,07/29/2023 17:30,07/29/2023,1730.0,07/31/2023 12:08,07/31/2023,1208.0,OTHER / UNKNOWN,NaN,HE,8,NaN,NaN,240,Auto Theft,NaN
2522582,20165028813,FAILURE TO REG AS SEX OFFENDER,1799,N,07/13/2016 11:29,07/13/2016,1129.0,07/13/2016 11:29,07/13/2016,1129.0,OTHER / UNKNOWN,NaN,NaN,NaN,N,07/14/2016,NaN,NaN,NaN
2522583,20115031193,PROTECTIVE ORDER,3829,N,06/23/2011 15:37,06/23/2011,1537.0,06/23/2011 15:37,06/23/2011,1537.0,RESIDENCE / HOME,NaN,DA,4,NaN,NaN,NaN,NaN,NaN


In [120]:
# Clean up Austin crime data to only include relevant columns: Date as datetime, and Offense
# Fill in missing values and display the first 5 rows of the cleaned data

austin_crime_df = austin_df[['Occurred Date', 'Highest Offense Description', "Highest Offense Code"]]

# Rename columns
austin_crime_df.columns = ['Date', 'Offense', 'Code']

# Convert 'Occurred Date' to datetime format and ensure consistency (set day as smallest increment)
austin_crime_df['Date'] = pd.to_datetime(austin_crime_df['Date']).dt.floor('D')

# Check for cleaned Austin data
austin_crime_df.info()
austin_crime_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522585 entries, 0 to 2522584
Data columns (total 3 columns):
 #   Column   Dtype         
---  ------   -----         
 0   Date     datetime64[ns]
 1   Offense  object        
 2   Code     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 57.7+ MB


/var/folders/vt/jy8dcmxj29vfz95p7lls9h1m0000gn/T/ipykernel_41001/2997512957.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  austin_crime_df['Date'] = pd.to_datetime(austin_crime_df['Date']).dt.floor('D')


,Date,Offense,Code
0,2003-01-29,FAMILY DISTURBANCE,3400
1,2003-01-01,DEADLY CONDUCT,408
2,2003-01-01,BURGLARY NON RESIDENCE,502
3,2003-01-01,DEADLY CONDUCT,408
4,2003-01-01,RESISTING ARREST OR SEARCH,905
...,...,...,...
2522580,2024-08-11,POSSESSION OF MARIJUANA,1803
2522581,2023-07-29,AUTO THEFT,700
2522582,2016-07-13,FAILURE TO REG AS SEX OFFENDER,1799
2522583,2011-06-23,PROTECTIVE ORDER,3829


In [121]:
#moon_df = moon_df.drop(columns=["Time (Universal Time)"])
moon_df.iloc[400]

Moon Phase                      Last Quarter
Date                     2015-01-13 00:00:00
Time (Universal Time)                  09:46
Name: 400, dtype: object

In [122]:
max_moon_date = moon_df["Date"].max()
min_moon_date = moon_df["Date"].min()
austin_crime_df = austin_crime_df[
    (austin_crime_df["Date"] <= max_moon_date) &
    (austin_crime_df["Date"] >= min_moon_date)]

austin_crime_df.shape

(1000152, 3)

In [123]:
moon_limited = moon_df.drop_duplicates(subset=["Date"])
moon_limited.shape

(495, 3)

In [124]:
# Merge dataframes and fill absent moon phase values with "Other"
merged_Austin_df = austin_crime_df.merge(
    moon_limited,
    how="left",
    on="Date"
).fillna("Other")

#Display the first 5 rows of the merged data
merged_Austin_df

,Date,Offense,Code,Moon Phase,Time (Universal Time)
0,2014-02-04,DEL CONTROLLED SUB/NARCOTIC,1804,Other,Other
1,2014-01-16,BURGLARY OF VEHICLE,601,Full Moon,04:52
2,2014-01-16,THEFT,600,Full Moon,04:52
3,2014-01-16,HARASSMENT,2703,Full Moon,04:52
4,2014-01-16,POSS CONTROLLED SUB/NARCOTIC,1800,Full Moon,04:52
...,...,...,...,...,...
1000147,2014-11-07,ASSAULT BY CONTACT FAM/DATING,902,Other,Other
1000148,2022-12-28,FRAUD - OTHER,1199,Other,Other
1000149,2018-03-02,PROTECTIVE ORDER,3829,Full Moon,00:51
1000150,2023-07-29,AUTO THEFT,700,Other,Other


In [125]:
merged_Austin_df["Moon Phase"].value_counts()

Moon Phase
Other            864728
New Moon          34103
Last Quarter      33802
First Quarter     33765
Full Moon         33754
Name: count, dtype: int64

In [126]:
merged_Austin_df["Moon Phase"] = merged_Austin_df["Moon Phase"].replace(["New Moon", "Last Quarter", "First Quarter"], "Other")
merged_Austin_df["Moon Phase"].value_counts()

Moon Phase
Other        966398
Full Moon     33754
Name: count, dtype: int64

In [127]:
# Create a dictionary of full moon and other counts for total rows (i.e. individual crimes)
moon_phase_crimes = merged_Austin_df["Moon Phase"].value_counts().to_dict()

In [128]:
# Get a total count for non-full-moon dates (i.e. days with "Other")
crimes_other_moon = merged_Austin_df[merged_Austin_df["Moon Phase"] == "Other"]
days_other_moon = crimes_other_moon["Date"].nunique()
days_other_moon

3524

In [129]:

crimes_full_moon = merged_Austin_df[merged_Austin_df["Moon Phase"] == "Full Moon"]
days_full_moon = crimes_full_moon["Date"].nunique()
days_full_moon

124

In [130]:
# Get a total count for individual crimes in each condition
full_moon_crimes = moon_phase_crimes.get("Full Moon")
other_moon_crimes = moon_phase_crimes.get("Other")
full_moon_crimes, other_moon_crimes 


(33754, 966398)

In [131]:
# Do the math for crime rate and percentage increase
full_crime_rate = full_moon_crimes / days_full_moon
other_crime_rate = other_moon_crimes / days_other_moon
percent_increase = (full_crime_rate / other_crime_rate - 1) * 100

print(f"The crime rate during a full moon is {full_crime_rate:.2f} against a control rate of {other_crime_rate:.2f}.")
print(f"These initial findings indicate a increased crime rate of {percent_increase:.2f}% above the base line.")

The crime rate during a full moon is 272.21 against a control rate of 274.23.
These initial findings indicate a increased crime rate of -0.74% above the base line.


In [132]:
# This looks at serious crimes (felonies, I think) by sorting them by offense code
serious_crimes_df = merged_Austin_df[merged_Austin_df["Code"] < 1000]
serious_crimes_df

,Date,Offense,Code,Moon Phase,Time (Universal Time)
1,2014-01-16,BURGLARY OF VEHICLE,601,Full Moon,04:52
2,2014-01-16,THEFT,600,Full Moon,04:52
5,2014-01-16,THEFT,600,Full Moon,04:52
9,2014-01-16,ASSAULT W/INJURY-FAM/DATE VIOL,900,Full Moon,04:52
11,2014-01-16,BURGLARY NON RESIDENCE,502,Full Moon,04:52
...,...,...,...,...,...
1000142,2018-10-05,ASSAULT BY CONTACT FAM/DATING,902,Other,Other
1000144,2015-12-15,ASSAULT W/INJURY-FAM/DATE VIOL,900,Other,Other
1000146,2021-12-17,THEFT FROM PERSON,610,Other,Other
1000147,2014-11-07,ASSAULT BY CONTACT FAM/DATING,902,Other,Other


In [133]:
# Getting values for felonies between both full moon and control
moon_phase_felonies = serious_crimes_df["Moon Phase"].value_counts().to_dict()
moon_phase_felonies

{'Other': 468000, 'Full Moon': 16327}

In [134]:
full_moon_felonies = moon_phase_felonies.get("Full Moon")
other_moon_felonies = moon_phase_felonies.get("Other")
full_moon_felonies, other_moon_felonies

(16327, 468000)

In [135]:
full_felony_rate = full_moon_felonies / days_full_moon
other_felony_rate = other_moon_felonies / days_other_moon
percent_increase_f = (full_felony_rate / other_felony_rate - 1) * 100

print(f"The felony rate during a full moon is {full_felony_rate:.2f} against a control rate of {other_felony_rate:.2f}.")
print(f"These initial findings indicate a increased felony rate of {percent_increase_f:.2f}% above the base line.")

The felony rate during a full moon is 131.67 against a control rate of 132.80.
These initial findings indicate a increased felony rate of -0.85% above the base line.


In [136]:
serious_crimes_df["Offense"].value_counts().head(20)

Offense
BURGLARY OF VEHICLE                              89776
THEFT                                            84254
ASSAULT W/INJURY-FAM/DATE VIOL                   33923
AUTO THEFT                                       33817
THEFT BY SHOPLIFTING                             28322
ASSAULT WITH INJURY                              24966
BURGLARY OF RESIDENCE                            23229
BURGLARY NON RESIDENCE                           17525
ASSAULT BY CONTACT                               16968
THEFT OF BICYCLE                                 13151
ASSAULT BY THREAT                                10341
AGG ASSAULT                                       9413
BURGLARY OF VEH-NO SUSPECT/FU                     8422
ASSAULT BY CONTACT FAM/DATING                     8103
THEFT OF LICENSE PLATE                            6303
THEFT OF SERVICE                                  6035
AGG ASLT STRANGLE/SUFFOCATE                       5967
BURGLARY OF SHED/DETACHED GARAGE/STORAGE UNIT     5863
TH

In [137]:
# Same thing as above but for assaults. (Using both the full word and the abbrivaiton to capture all)
assault_crimes_df = serious_crimes_df[serious_crimes_df["Offense"].str.contains("ASSAULT|ASLT")]
assault_crimes_df["Offense"].value_counts()

Offense
ASSAULT W/INJURY-FAM/DATE VIOL                 33923
ASSAULT WITH INJURY                            24966
ASSAULT BY CONTACT                             16968
ASSAULT BY THREAT                              10341
AGG ASSAULT                                     9413
ASSAULT BY CONTACT FAM/DATING                   8103
AGG ASLT STRANGLE/SUFFOCATE                     5967
AGG ASSAULT FAM/DATE VIOLENCE                   3947
ROBBERY BY ASSAULT                              3205
ASSAULT BY THREAT FAM/DATING                    2341
ASSAULT  CONTACT-SEXUAL NATURE                  1640
ASSAULT ON PUBLIC SERVANT                       1286
AGG ASLT ENHANC STRANGL/SUFFOC                   643
AGG ASSAULT WITH MOTOR VEH                       605
BURG OF RES - FAM/DATING ASLT                    360
ASSAULT ON PEACE OFFICER                         358
ASSAULT OF PREGNANT WM-FAM/DAT                   291
AGG ASLT W/MOTOR VEH FAM/DAT V                   288
AGG ROBBERY BY ASSAULT                

In [138]:
moon_phase_assault = assault_crimes_df["Moon Phase"].value_counts().to_dict()
moon_phase_assault

{'Other': 120935, 'Full Moon': 4157}

In [139]:
full_moon_assault = moon_phase_assault.get("Full Moon")
other_moon_assault = moon_phase_assault.get("Other")
full_moon_assault, other_moon_assault

(4157, 120935)

In [140]:
full_assault_rate = full_moon_assault / days_full_moon
other_assault_rate = other_moon_assault / days_other_moon
percent_increase_a = (full_assault_rate / other_assault_rate - 1) * 100

print(f"The assault rate during a full moon is {full_assault_rate:.2f} against a control rate of {other_assault_rate:.2f}.")
print(f"These initial findings indicate an increased assault rate of {percent_increase_a:.2f}% above the baseline.")

The assault rate during a full moon is 33.52 against a control rate of 34.32.
These initial findings indicate an increased assault rate of -2.31% above the baseline.


In [141]:
merged_Austin_df

,Date,Offense,Code,Moon Phase,Time (Universal Time)
0,2014-02-04,DEL CONTROLLED SUB/NARCOTIC,1804,Other,Other
1,2014-01-16,BURGLARY OF VEHICLE,601,Full Moon,04:52
2,2014-01-16,THEFT,600,Full Moon,04:52
3,2014-01-16,HARASSMENT,2703,Full Moon,04:52
4,2014-01-16,POSS CONTROLLED SUB/NARCOTIC,1800,Full Moon,04:52
...,...,...,...,...,...
1000147,2014-11-07,ASSAULT BY CONTACT FAM/DATING,902,Other,Other
1000148,2022-12-28,FRAUD - OTHER,1199,Other,Other
1000149,2018-03-02,PROTECTIVE ORDER,3829,Full Moon,00:51
1000150,2023-07-29,AUTO THEFT,700,Other,Other


In [149]:
# In other cities, trespassing looked to be a possible area to investigate.
trespassing_df = merged_Austin_df[merged_Austin_df["Offense"].str.contains("TRESPASS")]
trespassing_df

,Date,Offense,Code,Moon Phase,Time (Universal Time)
56,2014-01-16,CRIMINAL TRESPASS,2716,Full Moon,04:52
280,2014-01-17,CRIMINAL TRESPASS,2716,Other,Other
451,2014-01-19,CRIMINAL TRESPASS/HOTEL,2722,Other,Other
833,2014-01-20,CRIMINAL TRESPASS,2716,Other,Other
1095,2014-01-21,CRIMINAL TRESPASS,2716,Other,Other
...,...,...,...,...,...
998734,2021-05-13,CRIMINAL TRESPASS,2716,Other,Other
998850,2019-12-23,CRIMINAL TRESPASS/IN VEHICLE,2727,Other,Other
999077,2018-07-18,CRIMINAL TRESPASS,2716,Other,Other
999237,2016-09-18,CRIMINAL TRESPASS,2716,Other,Other


In [143]:
merged_Austin_df["Date"].nunique()

3648

In [144]:
trespassing_counts = trespassing_df["Moon Phase"].value_counts().to_dict()
trespassing_counts

{'Other': 18855, 'Full Moon': 663}

In [145]:
trespassing_full = trespassing_counts.get("Full Moon")
trespassing_other = trespassing_counts.get("Other")
trespassing_full, trespassing_other

(663, 18855)

In [146]:
# This data shows no relationship for trespassing either.
full_tres_rate = trespassing_full / days_full_moon
other_tres_rate = trespassing_other / days_other_moon
percent_increase_tres = (full_crime_rate / other_crime_rate - 1) * 100

print(f"The crime rate during a full moon is {full_tres_rate:.2f} against a control rate of {other_tres_rate:.2f}.")
print(f"These initial findings indicate a increased crime rate of {percent_increase_tres:.2f}% above the base line.")

The crime rate during a full moon is 5.35 against a control rate of 5.35.
These initial findings indicate a increased crime rate of -0.74% above the base line.
